In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13645,2021-12-04T17:00:00,ITA,19,Sicilia,38.115697,13.362357,300,45,345,12500,...,DECEDUTI: N. 1 IL 03/12/21 - N. 4 IL 02/12/21,3.0,NaN,NaN,327011.0,0.0,3569318.0,3973580.0,ITG,ITG1
13646,2021-12-04T17:00:00,ITA,9,Toscana,43.769231,11.255889,246,49,295,9747,...,NaN,2.0,NaN,NaN,298514.0,5542.0,5081255.0,3018711.0,ITI,ITI1
13647,2021-12-04T17:00:00,ITA,10,Umbria,43.106758,12.388247,39,8,47,1741,...,"Si fa presente che uno dei ricoveri NON UTI, n...",0.0,NaN,NaN,68028.0,0.0,1256661.0,1257963.0,ITI,ITI2
13648,2021-12-04T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,22,2,24,717,...,NaN,0.0,NaN,NaN,12242.0,1194.0,118488.0,177542.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13645,2021-12-04T17:00:00,19,Sicilia,300,45,345,12500,12845,23,549,...,327011,7542898,2765901.0,3.0,327011.0,0.0,3569318.0,3973580.0,ITG,ITG1
13646,2021-12-04T17:00:00,9,Toscana,246,49,295,9747,10042,150,677,...,304056,8099966,3361639.0,2.0,298514.0,5542.0,5081255.0,3018711.0,ITI,ITI1
13647,2021-12-04T17:00:00,10,Umbria,39,8,47,1741,1788,44,143,...,68028,2514624,475403.0,0.0,68028.0,0.0,1256661.0,1257963.0,ITI,ITI2
13648,2021-12-04T17:00:00,2,Valle d'Aosta,22,2,24,717,741,20,67,...,13436,296030,98418.0,0.0,12242.0,1194.0,118488.0,177542.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-12-04', '2021-12-03')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-04T17:00:00,1,Piemonte,419,37,456,12469,12925,685,1110,...,403693,9697620,2780641.0,6.0,371758.0,31935.0,3879727.0,5817893.0,ITC,ITC1
1,2021-12-04T17:00:00,2,Valle d'Aosta,22,2,24,717,741,20,67,...,13436,296030,98418.0,0.0,12242.0,1194.0,118488.0,177542.0,ITC,ITC2
2,2021-12-04T17:00:00,3,Lombardia,924,121,1045,35949,36994,1610,2990,...,943466,21089472,6216514.0,12.0,860305.0,83161.0,12075697.0,9013775.0,ITC,ITC4
3,2021-12-04T17:00:00,5,Veneto,573,118,691,36313,37004,1299,2560,...,528363,16894939,2321731.0,6.0,509223.0,19140.0,7696218.0,9198721.0,ITH,ITH3
4,2021-12-04T17:00:00,6,Friuli Venezia Giulia,281,25,306,7535,7841,217,829,...,134176,3950262,913823.0,3.0,116906.0,17270.0,2455535.0,1494727.0,ITH,ITH4
5,2021-12-04T17:00:00,7,Liguria,170,25,195,4812,5007,123,490,...,124617,3090440,972613.0,4.0,124617.0,0.0,1839249.0,1251191.0,ITC,ITC3
6,2021-12-04T17:00:00,8,Emilia-Romagna,703,77,780,23457,24237,695,1610,...,461740,10655900,2267249.0,6.0,461294.0,446.0,6699014.0,3956886.0,ITH,ITH5
7,2021-12-04T17:00:00,9,Toscana,246,49,295,9747,10042,150,677,...,304056,8099966,3361639.0,2.0,298514.0,5542.0,5081255.0,3018711.0,ITI,ITI1
8,2021-12-04T17:00:00,10,Umbria,39,8,47,1741,1788,44,143,...,68028,2514624,475403.0,0.0,68028.0,0.0,1256661.0,1257963.0,ITI,ITI2
9,2021-12-04T17:00:00,11,Marche,100,26,126,5162,5288,67,444,...,125693,1785765,1032489.0,1.0,125693.0,0.0,1511382.0,274383.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-03T17:00:00,1,Piemonte,413,34,447,11793,12240,705,1189,...,402583,9625959,2772444.0,1.0,371222.0,31361.0,3871000.0,5754959.0,ITC,ITC1
1,2021-12-03T17:00:00,2,Valle d'Aosta,24,2,26,695,721,-17,48,...,13369,293490,98135.0,1.0,12200.0,1169.0,118254.0,175236.0,ITC,ITC2
2,2021-12-03T17:00:00,3,Lombardia,912,114,1026,34358,35384,707,2809,...,940476,20952470,6199148.0,5.0,857315.0,83161.0,12043398.0,8909072.0,ITC,ITC4
3,2021-12-03T17:00:00,5,Veneto,549,115,664,35041,35705,1877,3116,...,525803,16799453,2316455.0,21.0,506890.0,18913.0,7675014.0,9124439.0,ITH,ITH3
4,2021-12-03T17:00:00,6,Friuli Venezia Giulia,284,25,309,7315,7624,160,820,...,133351,3924360,911503.0,1.0,116172.0,17179.0,2447366.0,1476994.0,ITH,ITH4
5,2021-12-03T17:00:00,7,Liguria,164,22,186,4698,4884,316,579,...,124127,3073127,970185.0,2.0,124127.0,0.0,1834136.0,1238991.0,ITC,ITC3
6,2021-12-03T17:00:00,8,Emilia-Romagna,687,75,762,22780,23542,1035,1633,...,460131,10615842,2262399.0,8.0,459689.0,442.0,6676536.0,3939306.0,ITH,ITH5
7,2021-12-03T17:00:00,9,Toscana,246,48,294,9598,9892,482,709,...,303379,8064780,3351443.0,4.0,297862.0,5517.0,5070759.0,2994021.0,ITI,ITI1
8,2021-12-03T17:00:00,10,Umbria,44,9,53,1691,1744,103,153,...,67885,2503044,474683.0,0.0,67885.0,0.0,1253954.0,1249090.0,ITI,ITI2
9,2021-12-03T17:00:00,11,Marche,98,25,123,5098,5221,20,476,...,125249,1779245,1028573.0,0.0,125249.0,0.0,1507084.0,272161.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-12-04T17:00:00,1,Piemonte,419,37,456,12469,12925,685,1110,...,9697620,2780641.0,6.0,371758.0,31935.0,3879727.0,5817893.0,ITC,ITC1,4.341375e+06
1,2021-12-04T17:00:00,2,Valle d'Aosta,22,2,24,717,741,20,67,...,296030,98418.0,0.0,12242.0,1194.0,118488.0,177542.0,ITC,ITC2,1.255010e+05
2,2021-12-04T17:00:00,3,Lombardia,924,121,1045,35949,36994,1610,2990,...,21089472,6216514.0,12.0,860305.0,83161.0,12075697.0,9013775.0,ITC,ITC4,1.010397e+07
3,2021-12-04T17:00:00,5,Veneto,573,118,691,36313,37004,1299,2560,...,16894939,2321731.0,6.0,509223.0,19140.0,7696218.0,9198721.0,ITH,ITH3,4.907704e+06
4,2021-12-04T17:00:00,6,Friuli Venezia Giulia,281,25,306,7535,7841,217,829,...,3950262,913823.0,3.0,116906.0,17270.0,2455535.0,1494727.0,ITH,ITH4,1.211357e+06
5,2021-12-04T17:00:00,7,Liguria,170,25,195,4812,5007,123,490,...,3090440,972613.0,4.0,124617.0,0.0,1839249.0,1251191.0,ITC,ITC3,1.543127e+06
6,2021-12-04T17:00:00,8,Emilia-Romagna,703,77,780,23457,24237,695,1610,...,10655900,2267249.0,6.0,461294.0,446.0,6699014.0,3956886.0,ITH,ITH5,4.467118e+06
7,2021-12-04T17:00:00,9,Toscana,246,49,295,9747,10042,150,677,...,8099966,3361639.0,2.0,298514.0,5542.0,5081255.0,3018711.0,ITI,ITI1,3.722729e+06
8,2021-12-04T17:00:00,10,Umbria,39,8,47,1741,1788,44,143,...,2514624,475403.0,0.0,68028.0,0.0,1256661.0,1257963.0,ITI,ITI2,8.802850e+05
9,2021-12-04T17:00:00,11,Marche,100,26,126,5162,5288,67,444,...,1785765,1032489.0,1.0,125693.0,0.0,1511382.0,274383.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-12-04T17:00:00,1,Piemonte,419,37,456,12469,12925,685,1110,...,5817893.0,ITC,ITC1,4.341375e+06,Piemonte,413,5824,34,628,99
1,2021-12-04T17:00:00,2,Valle d'Aosta,22,2,24,717,741,20,67,...,177542.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,24,99,2,33,0
2,2021-12-04T17:00:00,3,Lombardia,924,121,1045,35949,36994,1610,2990,...,9013775.0,ITC,ITC4,1.010397e+07,Lombardia,912,6688,114,1530,0
3,2021-12-04T17:00:00,5,Veneto,573,118,691,36313,37004,1299,2560,...,9198721.0,ITH,ITH3,4.907704e+06,Veneto,549,6000,115,1000,0
4,2021-12-04T17:00:00,6,Friuli Venezia Giulia,281,25,306,7535,7841,217,829,...,1494727.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,284,1277,25,175,0
5,2021-12-04T17:00:00,7,Liguria,170,25,195,4812,5007,123,490,...,1251191.0,ITC,ITC3,1.543127e+06,Liguria,164,1700,22,217,12
6,2021-12-04T17:00:00,8,Emilia-Romagna,703,77,780,23457,24237,695,1610,...,3956886.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,687,8336,75,889,0
7,2021-12-04T17:00:00,9,Toscana,246,49,295,9747,10042,150,677,...,3018711.0,ITI,ITI1,3.722729e+06,Toscana,246,5033,48,570,33
8,2021-12-04T17:00:00,10,Umbria,39,8,47,1741,1788,44,143,...,1257963.0,ITI,ITI2,8.802850e+05,Umbria,44,662,9,88,39
9,2021-12-04T17:00:00,11,Marche,100,26,126,5162,5288,67,444,...,274383.0,ITI,ITI3,1.518400e+06,Marche,98,966,25,250,57


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-12-04T17:00:00,3,Lombardia,924,121,1045,35949,36994,1610,2990,...,7.91,1591,181,137002,17366.0,19,1361,2.18,0.02959,9.34
1,2021-12-04T17:00:00,5,Veneto,573,118,691,36313,37004,1299,2560,...,11.80,1272,-556,95486,5276.0,8,1253,2.68,0.05216,10.77
2,2021-12-04T17:00:00,12,Lazio,712,91,803,22279,23082,1016,1832,...,9.65,1028,339,47671,8030.0,7,809,3.84,0.03123,7.34
3,2021-12-04T17:00:00,8,Emilia-Romagna,703,77,780,23457,24237,695,1610,...,8.66,677,-23,40058,4850.0,12,902,4.02,0.03604,10.34
4,2021-12-04T17:00:00,15,Campania,322,22,344,18200,18544,437,1216,...,4.23,426,-111,32948,3598.0,3,776,3.69,0.02102,8.57
5,2021-12-04T17:00:00,1,Piemonte,419,37,456,12469,12925,685,1110,...,5.89,676,-79,71661,8197.0,4,421,1.55,0.02557,9.30
6,2021-12-04T17:00:00,6,Friuli Venezia Giulia,281,25,306,7535,7841,217,829,...,14.29,220,9,25902,2320.0,5,603,3.20,0.06844,11.08
7,2021-12-04T17:00:00,9,Toscana,246,49,295,9747,10042,150,677,...,8.60,149,-32,35186,10196.0,2,525,1.92,0.01819,8.17
8,2021-12-04T17:00:00,19,Sicilia,300,45,345,12500,12845,23,549,...,5.24,34,-287,25952,5663.0,5,521,2.12,0.01105,6.58
9,2021-12-04T17:00:00,21,P.A. Bolzano,93,13,106,6246,6352,336,549,...,13.00,334,16,19344,1148.0,1,212,2.84,103.07072,16977.38


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Friuli Venezia Giulia,25,150,175,14.29
1,P.A. Bolzano,13,87,100,13.00
2,Veneto,118,882,1000,11.80
3,Liguria,25,192,217,11.52
4,Calabria,20,161,181,11.05
5,Marche,26,224,250,10.40
6,Lazio,91,852,943,9.65
7,Umbria,8,80,88,9.09
8,P.A. Trento,8,82,90,8.89
9,Emilia-Romagna,77,812,889,8.66


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))